## Whisper: Torch Compile for 4x Faster Inference

The Whisper model in Transformers is *overhead bound*: the bottle-neck in inference speed is the CPU not instructing the GPU fast enough for the GPU to be fully utilized. To address this, we need to provide the GPU with more operations at once. One way of achieving this is using [**torch compile**](https://pytorch.org/docs/stable/generated/torch.compile.html), a native PyTorch function for accelerating the inference speed of PyTorch models.

Torch compile takes a large region of a PyTorch graph and captures it into a single compiled region. In doing so, we can reduce the GPU instructions to this single compiled region, thus reducing the CPU overhead. Furthermore, torch compile generates faster kernels for the operations, speeding up computations and ensuring they are *memory bound*.

In this Colab, we'll see how torch compile can be enabled for the Whisper model with just two lines of code. We'll perform a benchmark that highlights the 4x inference speed improvements that torch compile provides to any Whisper model on the Hugging Face Hub.

## Set-Up

The runtime is already configured to use the free 16GB T4 GPU provided through Google Colab Free Tier, so all we need to do is hit the button `Connect T4` in the top right-hand corner of the screen.

Once we've done that, we can go ahead and install the necessary Python packages. We'll install [🤗 Transformers](https://huggingface.co/docs/transformers/index) for loading and running the Whisper models, [🤗 Datasets](https://huggingface.co/docs/datasets/index) for loading our benchmarking dataset, and [🤗 Accelerate](https://huggingface.co/docs/accelerate/index) for fast model loading:

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet transformers datasets[audio] accelerate

## Benchmarking

First, we'll load the Whisper model and its accompanying processor using the familiar 🤗 Transformers API.
In this example, we'll load the pre-trained Whisper [medium.en](https://huggingface.co/openai/whisper-medium.en) model, but you're free
to swap this for any one of the [10k Whisper checkpoints](https://huggingface.co/models?library=transformers&other=whisper&sort=trending) 
on the Hugging Face Hub. To reduce the loading time, we'll pass the [low_cpu_mem_usage](https://huggingface.co/docs/transformers/v4.43.4/en/big_models#accelerates-big-model-inference) flag to `.from_pretrained`:

In [2]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium.en", torch_dtype=torch_dtype, low_cpu_mem_usage=True)
model.to(device)

processor = WhisperProcessor.from_pretrained("openai/whisper-medium.en")

/home/sanchit/miniconda3/envs/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For benchmarking, we'll load a small dataset consisting of 73 samples from the [LibriSpeech ASR](https://huggingface.co/datasets/librispeech_asr) validation-clean dataset. This amounts to ~9MB of data, so it's very lightweight and quick to download on-device:

In [3]:
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
dataset

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

To ensure the sampling rate of the audios matches the sampling rate of our model, we'll re-sample the audios to the sampling rate expected by Whisper (16kHz).
Note that the re-sampling is applied on-the-fly when the audios are loaded, with a no-op if the sampling rate already matches:

In [4]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

We're ready to start benchmarking 📏 The following cell iterates over samples in our dataset one-by-one (i.e. with a batch size of one). For each sample, we perform three stages of inference:
1. Pre-processing the raw audio inputs to log-mel spectrograms
2. Auto-regressively generating the text tokens, conditional on the spectrogram inputs
3. Post-processing the generated tokens to text strings

For the purposes of benchmarking, we'll time the generation step, which is the portion performed by the Whisper model itself:

In [5]:
import time
from tqdm import tqdm
from torch.nn.attention import sdpa_kernel, SDPBackend

inference_time = 0.0
model.generation_config.max_new_tokens = 128

for sample in tqdm(dataset):
    # 1. Pre-process the audio inputs
    input_features = processor(sample["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features
    input_features = input_features.to(device, dtype=torch_dtype)
    
    # 2. Auto-regressively generate text tokens
    start = time.time()
    pred_ids = model.generate(input_features)
    inference_time += time.time() - start

    # 3. Post-process tokens to text
    pred_text = processor.batch_decode(pred_ids, skip_special_tokens=True)

print(inference_time)

100%|███████████████████████████████████████████████████████████████████████████████████| 73/73 [00:37<00:00,  1.95it/s]

35.18650460243225


We have a baseline of **35.2-seconds** for our un-compiled model. Let's now apply torch compile and re-measure performance.

## Enable torch compile

The first step in enabling compile is self-explanatory: we need to apply the `torch.compile` transformation to the model forward pass. We'll set the compile mode to `reduce-overhead`, which uses [CUDA Graphs](https://pytorch.org/blog/accelerating-pytorch-with-cuda-graphs/) to further reduce CPU overhead. We'll also set `fullgraph=True` to compile the entire model in one graph (i.e. with no graph breaks):

In [8]:
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

The second step involves setting the key-value (kv) cache. During decoding, the Whisper decoder computes the kv states for each new input token and saves them to be re-used at the next decoding step, forming a **kv-cache**. The default kv-cache implementation grows in length with each generated token, since we save a new set of kv states for each decoding step.

While dynamic shapes are compatible with a subset of `torch.compile` optimizations, they limit the extent to which the CPU overhead can be reduced. Thus, we'll switch the kv cache to a **static** implementation, which pre-allocates the entire kv-cache size to the maximum value and masks out the un-used parts from the attention computation. In doing so, this kv-cache implementation is compatible with the `reduce-overhead` setting from the previous step.

In [9]:
model.generation_config.cache_implementation = "static"

Since torch compile is a "just in time" (JIT) compilation, we need to perform a set of compilation steps to compile our model. Here, we'll perform three warm-up steps, generating to our maximum number of permitted tokens each time:

In [10]:
max_new_tokens = model.generation_config.max_new_tokens

for _ in tqdm(range(3)):
    with sdpa_kernel(SDPBackend.MATH):
        model.generate(input_features, min_new_tokens=max_new_tokens, max_new_tokens=max_new_tokens)

100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:51<00:00, 77.24s/it]


**Note:** this code-cell may take several minutes to run, particularly the first time it is called. To reduce the compilation time of subsequent runs, upgrade to `torch>2.4` and enable the [FX graph cache](https://pytorch.org/tutorials/recipes/torch_compile_caching_tutorial.html) with the flag `TORCHINDUCTOR_FX_GRAPH_CACHE=1`.

## Benchmarking with Compile

We're now ready to re-run our benchmark using the compiled implementation. The only change we'll make is using the [scaled dot product attention (SDPA) context manager](https://pytorch.org/docs/stable/generated/torch.nn.attention.sdpa_kernel.html) to switch the attention implementation from flash attention to the native PyTorch C++ implementation, which typically leads to better performance under compile:

In [11]:
inference_time = 0.0

for sample in tqdm(dataset):
    # 1. Pre-process the audio inputs
    input_features = processor(sample["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features
    input_features = input_features.to(device, dtype=torch_dtype)
    
    # 2. Auto-regressively generate text tokens
    start = time.time()
    with sdpa_kernel(SDPBackend.MATH):
        pred_ids = model.generate(input_features)
    inference_time += time.time() - start

    # 3. Post-process tokens to text
    pred_text = processor.batch_decode(pred_ids, skip_special_tokens=True)

print(inference_time)

100%|███████████████████████████████████████████████████████████████████████████████████| 73/73 [00:10<00:00,  7.03it/s]

8.95268702507019


The inference speed is reduced to just **9.0-seconds**, representing a speed-up of 3.9x with no degradation to model accuracy ⚡️

Recall that this optimization technique is model-agnostic: it can be applied to any Whisper model in the Transformers library. The largest speed-ups will typically be seen for smaller models. However, even the largest Whisper model ([large-v3](https://huggingface.co/openai/whisper-large-v3)) obtains >3x speed-up.

## Conclusion

In this Colab, we've broken down the steps for Whisper inference using torch compile, demonstrating a 3.5x speed-up from two lines of additional code. For an end-to-end code example, refer to the [Whisper model card](https://huggingface.co/openai/whisper-large-v3#torch-compile).